# 4.6 编码 GPT 模型

我们在本章开头用一个被称为 DummyGPTModel 的 GPT 架构的宏观概述开始。在这个 DummyGPTModel 代码实现中，我们展示了 GPT 模型的输入和输出，但其构建模块仍然是一个黑盒，使用 DummyTransformerBlock 和 DummyLayerNorm 类作为占位符。

在本节中，我们现在用我们在本章后面编码的真实的 TransformerBlock 和 LayerNorm 类替换掉 DummyTransformerBlock 和 DummyLayerNorm 占位符，组装一个完全工作的原始 1.24 亿参数版本的 GPT-2。在第 5 章，我们将预训练一个 GPT-2 模型，并在第 6 章加载 OpenAI 的预训练权重。

在代码中组装 GPT-2 模型之前，让我们看看其整体结构，如图 4.15 所示，它结合了本章迄今为止我们所涉及的所有概念。

**图 4.15 GPT 模型架构概览。该图示说明了数据通过 GPT 模型的流动。从底部开始，标记化文本首先被转换为标记嵌入，然后与位置嵌入增强。这些组合信息形成一个张量，通过中间显示的一系列变换器块传递（每个包含多头注意力和带有 dropout 和层标准化的前馈神经网络层），这些块堆叠在一起并重复 12 次。**

如图 4.15 所示，我们在第 4.5 节编码的变换器块在 GPT 模型架构中多次重复。在 1.24 亿参数的 GPT-2 模型中，这个变换器块重复了 12 次，我们通过在 GPT_CONFIG_124M 字典中指定 "n_layers" 条目。在参数最多的 GPT-2 模型中，即 1,542 亿参数，这个变换器块重复了 36 次。

![fig4.15](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-15.jpg?raw=true)

如图 4.15 所示，最后一个变换器块的输出然后通过最终的层标准化步骤，然后到达线性输出层。这个层将变换器的输出映射到高维空间（在这个例子中，是 50,257 维，对应于模型的词汇表大小）来预测序列中的下一个标记。

现在让我们在代码中实现我们在图 4.15 中看到的架构：

**清单 4.7 GPT 模型架构实现**


**Listing 4.7 The GPT model architecture implementation**

```
class GPTModel(nn.Module):
    def __init__(self, cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
    self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
    self.drop_emb = nn.Dropout(cfg["drop_rate"])
    self.trf_blocks = nn.Sequential(
    *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
    self.final_norm = LayerNorm(cfg["emb_dim"])
    self.out_head = nn.Linear(
    cfg["emb_dim"], cfg["vocab_size"], bias=False
    )
    def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    tok_embeds = self.tok_emb(in_idx)
    #A
    pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
    x = tok_embeds + pos_embeds
    x = self.drop_emb(x)
    x = self.trf_blocks(x)
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits
    

由于我们在第 4.5 节实现的 TransformerBlock 类，GPTModel 类相对较小且紧凑。

The \__init\__ constructor of this GPTModel class initializes the token and
positional embedding layers using the configurations passed in via a Python
dictionary, cfg. These embedding layers are responsible for converting input
token indices into dense vectors and adding positional information, as
discussed in chapter 2.

Next, the \__init\__ method creates a sequential stack of TransformerBlock
modules equal to the number of layers specified in cfg. Following the
transformer blocks, a LayerNorm layer is applied, standardizing the outputs
from the transformer blocks to stabilize the learning process. Finally, a linear
output head without bias is defined, which projects the transformer's output
into the vocabulary space of the tokenizer to generate logits for each token in
the vocabulary.

 forward 方法接收一批输入标记索引，计算它们的嵌入，应用位置嵌入，通过变换器块传递序列，标准化最终输出，然后计算 logits，表示下一个标记的未规范化概率。我们将在下一节中将这些 logits 转换为标记和文本输出。

现在，让我们使用我们传入 cfg 参数的 GPT_CONFIG_124M 字典初始化 1.24 亿参数的 GPT 模型，并用我们在本章开头创建的批量文本输入来喂养它：

````
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

前面的代码打印了输入批次的内容，然后是输出张量：

输入批次：

````
tensor([[ 6109, 3626, 6100, 345], # token IDs of text 1
[ 6109, 1110, 6622, 257]]) # token IDs of text 2

输出规模:

````
torch.Size([2, 4, 50257])
tensor([[[ 0.3613, 0.4222, -0.0711, ..., 0.3483, 0.4661, -0.2838],
[-0.1792, -0.5660, -0.9485, ..., 0.0477, 0.5181, -0.3168],
[ 0.7120, 0.0332, 0.1085, ..., 0.1018, -0.4327, -0.2553],
[-1.0076, 0.3418, -0.1190, ..., 0.7195, 0.4023, 0.0532]],
[[-0.2564, 0.0900, 0.0335, ..., 0.2659, 0.4454, -0.6806],
[ 0.1230, 0.3653, -0.2074, ..., 0.7705, 0.2710, 0.2246],
[ 1.0558, 1.0318, -0.2800, ..., 0.6936, 0.3205, -0.3178],
[-0.1565, 0.3926, 0.3288, ..., 1.2630, -0.1858, 0.0388]]],
grad_fn=<UnsafeViewBackward0>)

正如我们所见，输出张量的形状为 [2, 4, 50257]，因为我们传入了 2 个输入文本，每个文本 4 个标记。最后一个维度，50,257，对应于分词器的词汇表大小。在下一节中，我们将看到如何将这些 50,257 维的输出向量转换回标记。

在我们继续下一节并编写将模型输出转换为文本的函数之前，让我们花更多时间研究模型架构本身并分析其大小。

使用 `numel()` 方法，即“元素数量”的缩写，我们可以收集模型参数张量中的总参数数量：

````
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

结果如下:

````
Total number of parameters: 163,009,536

现在，好奇的读者可能会注意到一个差异。早些时候，我们谈到了初始化一个1.24亿参数的GPT模型，那为什么实际的参数数量为1.63亿，如前面的代码输出所示？

原因是原始 GPT-2 架构中使用的一个称为权重共享（weight tying）的概念，这意味着原始 GPT-2 架构在其输出层中重用了标记嵌入层的权重。为了理解这意味着什么，让我们看一下我们之前通过 GPTModel 在模型上初始化的标记嵌入层和线性输出层的形状：

````
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

正如我们所基于打印输出看到的，这两个层的权重张量形状相同：

````
Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])

由于分词器词汇表中的 50,257 个条目，标记嵌入层和输出层非常大。根据权重共享的原则，让我们从 GPT-2 模型的总参数计数中去除输出层的参数计数：

````
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print(f"Number of trainable parameters considering weight tying:

输出的结果如下:

````
Number of trainable parameters considering weight tying: 124,412,1

正如我们所见，模型现在只有1.24亿参数，与原始GPT-2模型的大小相匹配。

权重共享可以减少模型的整体内存占用和计算复杂性。然而，根据我的经验，使用独立的标记嵌入层和输出层可以获得更好的训练效果和模型性能；因此，我们在我们的 GPTModel 实现中使用了独立的层。现代的大型语言模型（LLM）也是如此。不过，我们将在第6章重新访问并实现权重共享概念，届时我们将加载来自 OpenAI 的预训练权重。

**练习 4.1 前馈和注意力模块中的参数数量**

计算并比较包含在前馈模块中的参数数量和包含在多头注意力模块中的参数数量。

最后，让我们计算我们的 GPTModel 对象中 1.63 亿参数的内存需求：

````
total_size_bytes = total_params * 4 #A
total_size_mb = total_size_bytes / (1024 * 1024) #B
print(f"Total size of the model: {total_size_mb:.2f} MB")

输出的结果如下:

````
Total size of the model: 621.83 MB

结论是，通过计算我们GPTModel对象中的163百万参数的内存需求，并假设每个参数是一个占用4个字节的32位浮点数，我们发现模型的总大小为621.83 MB，这说明即使是相对较小的LLM也需要相当大的存储容量。

在这一部分，我们实现了GPTModel架构，并看到它输出形状为[batch_size, num_tokens, vocab_size]的数值张量。在接下来的部分中，我们将编写代码将这些输出张量转换为文本。

****Exercise 4.2 初始化更大的GPT模型****

在本章中，我们初始化了一个1.24亿参数的GPT模型，被称为“GPT-2 small”。除了更新配置文件之外，不做任何代码修改，使用GPTModel类来实现GPT-2 medium（使用1024维嵌入、24个变换器块、16个多头注意力头）、GPT-2 large（1280维嵌入、36个变换器块、20个多头注意力头）和GPT-2 XL（1600维嵌入、48个变换器块、25个多头注意力头）。作为额外的任务，计算每个GPT模型中的总参数数量。